# Взрывающиеся и исчезающие градиенты

В данном ноутбуке мы расскажем про то, какими проблемами обладают градиенты при обучении нейронных сетей, когда их можно встретить и как избежать.

In [ ]:
!pip install tensorflow

import tensorflow as tf

## Взрыв градиентов (exploding gradients, gradient burst)

### Что это?

Взрыв градиентов - это ситуация, когда на начальном этапе обучения вследствие достаточно больших ошибок сети градиенты накапливаются в достаточно значительном масштабе и при обратном распространении ошибки значения весов очень сильно изменяются. Вследствие стохастической природы алгоритма оптимизации, это может привести к тому, что сеть попадет в состояние с еще большей ошибкой (на другом краю ямы минимума в многомерном пространстве оптимизации) и последующий градиент будет еще большим. Сеть попадает в самразбалансирующийся процесс накопления градиентов, и они вскоре достигают огромных размеров. Обычно это выражается в нестабильном обучении (если взрыв градиентов проходит) или вообще в невозможности обучить сеть.  

### Где можно встретить взрыв градиентов?
Чаще всего взрыв градиентов можно встретить вследствие ошибок в реализации модулей, градиентного спуска или функции потерь, если вы используете нестандартные модули или разрабатываете самостоятельно. Работая с стандартными модулями и средствами создания нейронных сетей, чаще всего взрыв градиентов встречается при работе с рекуррентными сетями и генеративными состязательными сетями.

### Как понять, что происходит взрыв градиентов?
- Значение функции потерь очень быстро растет
- Значение функции потерь стало равно NaN
- Веса слоев приобрели очень большие значения

### Как предотвратить взрыв градиентов?
В первую очередь, в случае использования нестандартных модулей, стоит проверить их реализацию с использованием отладчика. В случае же использования стандартных модулей можно:
- В случае обучения рекуррентных моделей использовать LSTM или GRU слои вместо простых рекуррентных слоев
- Уменьшить размер батча, снизив т.о. величину градиента на каждом шаге
- Использовать gradient clipping (см. ниже)
- Использовать регуляризацию весов в слое (см. ниже)
- Уменьшить значение learning rate

Регуляризация слоев возможна с помощью модуля `tf.keras,regularizers`. В нем содержатся стандартные имплементации знакомых вам L1 и L2 регуляризационных техник, которые можно использовать для слоев нейронной сети.

Также в модуле `tensorflow.keras.constraints` доступны ограничения на веса. С помощью данных техник можно поставить ограничения на веса слоя (по максимальному значению веса нейрона, запретить отрицательные значения, и т.д.

In [ ]:
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.constraints import MaxNorm

layer = tf.keras.layers.Dense(
    32,
    kernel_regularizer=l1(0.01),
    bias_regularizer=l2(0.02),
    activity_regularizer=l1_l2(0.01),
    
    kernel_constraint=MaxNorm(2),
    bias_constraint=MaxNorm(2)
)

Gradient Clipping - это техника ограничения максимального значения градиента. Все значения больше, чем максимальное, уменьшаются до максимального.
Ограничение градиента реализуется в любом оптимизаторе с помощью параметра clipvalue (ограничение максимального значения) или clipnorm (ограничение нормы).

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(
    learning_rate=0.01,
    clipnorm=1.0,
    clipvalue=3.0
)

---

## Затухание градиента

### Что такое затухание градиента?
Затухание градиента - гораздо более частая проблема, которую можно встретить при обучении нейронных сетей. Чаще всего это проблема касается не всей сети в целом, а отдельных ее частей. Суть затухания градиента заключается в том, что по разным причинам градиент на текущем участке становится настолько мал, что лишь незначительно изменяет веса на данном участке. Это может происходить по разным причинам, но обычно результатом этого является очень медленный (или остановившийся совсем) процесс обучения нейронной сети.

### Почему происходит затухание градиента?
Есть множество разных причин, по которым градиент может затухать.
#### Функции активации
Некоторые функции активации (такие, как tanh или sigmoid) ограничивают значение в отрезке около 0 ($[-1; +1]$ и $[0; 1]$ соответственно). При использовании нескольких слоев, каждый последующий слой в процессе обратного распространения ошибки (т.е. более близкий к началу сети) будет получать все меньшие значения градиентов, т.о. слои ближе к началу будут медленнее обновлять свои веса в процессе оптимизации.
#### Глубина сети
При использовании очень глубоких сетей градиент может затухать вне зависимости от выбора функции активации, т.к. его абсолютное значение уменьшается в процессе обратного распространения.
#### Использование рекуррентных сетей
В рекуррентных сетях вследствие необходимости подсчета градиента по всей цепочке данных в обратную сторону, проблема затухающих градиентов также встречается достаточно часто.

### Как предотвратить затухание градиента?
- Используйте альтернативные функции активации. Такие функции, как ReLU, LeakyReLU слабее ограничивают значение градиента и способствуют более легкому его распространению
    - В общем случае рекомендуется использовать LeakyReLU вместо ReLU, т.к. последний обладает неприятным свойством обнуления нейрона. Т.к. производная данной функции активации равна 0 при отрицательном значении нейрона, если нейрон будет выдавать отрицательные значения, то его процесс обучения остановится и градиенты больше не будут его обновлять. Данный нейрон будет "мертвым" и т.о. емкость вашей сети снижается.
- Используйте архитектуры сетей, в которых присутствуют skip connections. Такие соединения позволяют градиенту более свободно проходить до начальных слоев. Изобретение сети ResNet, в которой данный подход был представлен, позволило обучать гораздо более глубокие сети, нежели до этого. Некоторые современные исследования (см. https://habr.com/ru/post/351924/) показывают, что кажется, что нет причин вообще _**не**_ использовать skip connections в нейронных сетях.
- В случае использования рекуррентных сетей используйте модули LSTM и GRU, которые позволяют градиенту более свободно течь в процессе обучения, нежели стандартные рекуррентные слои.